In [13]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch.cuda import FloatTensor

# Req for package
sys.path.append("../")
from SkinLearning.NN.Helpers import train, test, DEVICE, getParameterLoss, setSeed
from SkinLearning.Utils.Dataset import getDataset, getSplit
from SkinLearning.Utils.Plotting import plotParameterBars


torch.backends.cudnn.benchmark = True

In [14]:
setSeed()

In [15]:
# Remove one FC LAyer
class MultiLSTM(nn.Module):
    def __init__(self, hidden_size=256, single_fc=True, out="f_hidden"):
        super(MultiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.out = out
        
        self.conv1 = nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False)
        self.pool1 = nn.MaxPool1d(kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm1d(128)
        
        self.conv2 = nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.conv3 = nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.bn3 = nn.BatchNorm1d(512)
        
        self.lstm = nn.LSTM(15, hidden_size, batch_first=True, num_layers=1, bidirectional=True)
        
        input_tensor = torch.zeros(32, 512, 15)
        output, hidden = self.lstm(input_tensor)
        
        fc_in = hidden_size
        if out == 'output':
            fc_in = output.shape[1] * output.shape[2]
        elif out == 'f-output':
            fc_in = output.shape[2]
        elif fc_in == 'hidden' or out == 'cell':
            out = hidden_size * output.shape[2]
        elif out == 'f-hiden' or out == 'f-cell':
            fc_in = output.shape[2]
        elif out == 'h+o' or out == 'h+c' :
            fc_in = output.shape[1]
        
        if single_fc:
            self.fc = nn.Linear(fc_in*2, 6)
        else:
            self.fc = nn.Sequential(
                nn.Linear(fc_in*2, 128),
                nn.ReLU(),
                nn.Linear(128 , 64),
                nn.ReLU(),
                nn.Linear(64, 6),   
            )

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.pool1(torch.relu(self.bn1(self.conv1(x))))
        x = self.pool2(torch.relu(self.bn2(self.conv2(x))))
        x = self.pool3(torch.relu(self.bn3(self.conv3(x))))
        
        #h0 = torch.zeros(1, batch_size, 256).to(x.device)
        #c0 = torch.zeros((1, batch_size, self.hidden_size)).to(x.device)
        o, (h, c) = self.lstm(x)
        
        if self.out == "f_hidden":
            x = h[-1].reshape(batch_size, -1)
        elif self.out == "hidden":
            x = h.reshape(batch_size, -1)
        elif self.out == "f_output":
            x = o[:, -1, :].reshape(batch_size, -1)
        elif self.out == "output":
            x = o.reshape(batch_size, -1)
        elif self.out == "f_cell":
            x == c[-1].reshape(batch_size, -1)
        elif self.out == "cell":
            x == c.reshape(batch_size, -1)
        elif self.out == "h+c":
            x = torch.concat([h[-1], c[-1]], dim=1).view(batch_size, -1)
        elif self.out == "h+o":
            x = torch.concat([h[-1], o[:, -1, :]], dim=1).view(o.size(0), -1)
            
        x = self.fc(x)
        return x

In [34]:
def test(test_loader, net, scaler, cluster=True):
    net.to(DEVICE)
    net.eval()
    criterion = nn.L1Loss()
    acc = 0
    p_acc = 0
    mae = []

    def testBatch(it):
        for _, data in enumerate(it):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)
                predicted = net(inp)
                
                # Denormalise
                p = scaler.inverse_transform(predicted.cpu().numpy())
                o = scaler.inverse_transform(out.cpu().numpy())
                    
                # Get column wise and overall MAPE
                # Since each column is normalised should also be able to use MAE*100
                p_mape = np.abs(p-o)/o
                #p_mape = np.mean(p_mape, axis=0)
                p_accuracy = p_mape#(1 - p_mape)*100

                mape = np.abs(p-o)/o
                #mape = np.mean(mape)
                accuracy = mape#(1 - mape)*100

                mae.append(criterion(predicted, out).item())
            
                p_acc += p_accuracy
                acc += accuracy

    with torch.no_grad():
        if not cluster:
            with tqdm(test_loader, unit=" batch") as it:
                testBatch(it)
        else:
            testBatch(test_loader)
            
    
    average_mape = acc/len(dataset)
    average_p_loss = np.mean(p_acc, axis=0)
    mae_mean = np.mean(mae)
    
    return average_mape, average_p_loss, mae_mean

In [17]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:13<00:00, 165.03it/s]


In [18]:
train_loader, test_loader = getSplit(dataset)

In [19]:
test1 = MultiLSTM(out="f_output", single_fc=False)

In [7]:
traint_loss, valt_loss = train(train_loader, test, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)

Using: cuda:0


Epoch 136/1500: 100%|███████████████████| 56/56 [00:02<00:00, 27.14batch/s, counter=9, lastLoss=0.0621, valLoss=0.0644]


Epoch 272/1500: 100%|███████████████████| 56/56 [00:02<00:00, 27.07batch/s, counter=8, lastLoss=0.0501, valLoss=0.0738]


Epoch 408/1500: 100%|██████████████████| 56/56 [00:03<00:00, 15.04batch/s, counter=20, lastLoss=0.0379, valLoss=0.0535]


Epoch 437/1500: 100%|██████████████████| 56/56 [00:03<00:00, 14.79batch/s, counter=49, lastLoss=0.0351, valLoss=0.0437]


Early stopping after 437 epochs
Average train loss: 0.01652565779151129
Average validation loss: 0.017377336887249517


In [35]:
test(test_loader, test1, scaler)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.

In [9]:
def test2(test_loader, net, scaler, earlyStopping=True):
    net.to(DEVICE)
    net.eval()
    criterion = nn.L1Loss()

    losses = []
    p_losses = []
    mae = []

    with torch.no_grad():
        with tqdm(test_loader, unit=" batch") as it:
            for idx, data in enumerate(it):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = net(inp)
                
                # Denormalise
                p = scaler.inverse_transform(predicted.cpu().numpy())
                o = scaler.inverse_transform(out.cpu().numpy())
                    
                # Get column wise and overall MAPE
                # Since each column is normalised should also be able to use MAE*100
                p_loss = np.mean(100*(np.abs(o-p)/o), axis=0)
                loss = np.mean(100*(np.abs(o-p)/o))

                mae.append(criterion(predicted, out).item())
        
            
                p_losses.append(p_loss)
                losses.append(loss)

            
    average_mape = 100 - np.mean(losses)
    average_p_loss = 100 - np.mean(p_losses, axis=0)
    mae_mean = np.mean(mae)
    
    return average_mape, average_p_loss, mae_mean

In [38]:
names = [
    "Output", "Hidden", "Final Hidden", "Final Output", "Final Output + Final Hidden"
]

single_fcs = [
    MultiLSTM(out="output"),
    MultiLSTM(out="hidden"),
    MultiLSTM(out="f_hidden"),
    MultiLSTM(out="f_output"),
    MultiLSTM(out="h+o"),
]

multi_fcs = [
    MultiLSTM(out="output", single_fc=False),
    MultiLSTM(out="hidden", single_fc=False),
    MultiLSTM(out="f_hidden", single_fc=False),
    MultiLSTM(out="f_output", single_fc=False),
    MultiLSTM(out="h+o", single_fc=False),
]

In [62]:
import pickle
with open("../Results/LSTMS_train_test", "wb") as f:
    pickle.dump([multi_fcs_train[3], multi_fcs_val[3], "Multi LSTM Final Output"], f)

In [46]:
single_fcs_train, single_fcs_val = [], []
multi_fcs_train, multi_fcs_val = [], []


"""for i, model in enumerate(single_fcs):
    print(f"Training single fc model {i}/{len(names)-1} using {names[i]}")
    train_loss, val_loss =train(train_loader, model, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)
    single_fcs_train.append(train_loss)
    single_fcs_val.append(val_loss)
    print("\n")"""

for i, model in enumerate(multi_fcs):
    print(f"Training multi fc model {i}/{len(names)-1} using {names[i]}")
    train_loss, val_loss = train(train_loader, model, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)
    multi_fcs_train.append(train_loss)
    multi_fcs_val.append(val_loss)
    print("\n")

Training multi fc model 0/4 using Output
Using: cuda:0


Epoch 136/1500: 100%|███████████████████| 56/56 [00:03<00:00, 14.05batch/s, counter=9, lastLoss=0.0451, valLoss=0.0513]


Epoch 272/1500: 100%|███████████████████| 56/56 [00:03<00:00, 14.08batch/s, counter=3, lastLoss=0.0319, valLoss=0.0391]


Epoch 359/1500: 100%|██████████████████| 56/56 [00:03<00:00, 17.04batch/s, counter=49, lastLoss=0.0311, valLoss=0.0417]


Early stopping after 359 epochs
Average train loss: 0.011105067873672981
Average validation loss: 0.012311908535038432


Training multi fc model 1/4 using Hidden
Using: cuda:0


...

Epoch 272/1500: 100%|██████████████████| 56/56 [00:04<00:00, 13.62batch/s, counter=32, lastLoss=0.0485, valLoss=0.0486]


Epoch 408/1500: 100%|███████████████████| 56/56 [00:04<00:00, 11.52batch/s, counter=5, lastLoss=0.0427, valLoss=0.0467]


Epoch 544/1500: 100%|██████████████████| 56/56 [00:12<00:00,  4.35batch/s, counter=42, lastLoss=0.0346, valLoss=0.0473]


Epoch 551/1500: 100%|██████████████████| 56/56 [00:09<00:00,  5.67batch/s, counter=49, lastLoss=0.0347, valLoss=0.0453]


Early stopping after 551 epochs
Average train loss: 0.020334804599572508
Average validation loss: 0.021504218585499463




In [36]:
# with init
# could be better for input with strong temporal structure
single_fcs_train, single_fcs_val = [], []
multi_fcs_train, multi_fcs_val = [], []


for i, model in enumerate(single_fcs):
    print(f"Training single fc model {i}/{len(names)} using {names[i]}")
    train_loss, val_loss =train(train_loader, model, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)
    single_fcs_train.append(train_loss)
    single_fcs_val.append(val_loss)
    print("\n")

for i, model in enumerate(multi_fcs):
    print(f"Training multi fc model {i}/{len(names)} using {names[i]}")
    train_loss, val_loss = train(train_loader, model, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)
    multi_fcs_train.append(train_loss)
    multi_fcs_val.append(val_loss)
    print("\n")

Training single fc model 0/5 using Output
Using: cuda:0


Epoch 43/1500:  96%|███████████████████▎| 54/56 [00:06<00:00,  8.99batch/s, counter=3, lastLoss=0.0687, valLoss=0.0738]


KeyboardInterrupt: 

In [48]:
def getParameterLoss(models, names, test_loader, scaler, output=False):
    params = []
    overall = []
    
    # Run evaluation on all models
    for model in models:
        avg, ps, _, = test(test_loader, model, scaler)
        overall.append(avg)
        params.append(ps)
    
    all_vals = np.array(params)
    df = pd.DataFrame({
        "Architecture": names,
        "YM (Skin)": all_vals[:, 0],
        "YM (Adipose)": all_vals[:, 1],
        "PR (Skin)": all_vals[:, 2],
        "PR (Adipose)": all_vals[:, 3],
        "Perm (Skin)": all_vals[:, 4],
        "Perm (Adipose)": all_vals[:, 5],
        "Overall": overall
    })
    
    df = df.set_index("Architecture")

    if output:
        df = df.style.set_caption(
            'Average percent correctness 100-MAPE').set_table_styles([{
            'selector': 'caption',
            'props': [
                ('color', 'black'),
                ('font-size', '16px'),
                ('text-align', 'center')
            ]
        }])

        display(df)
    else:
        return df

In [49]:
df = getParameterLoss(single_fcs, ["Single LSTM "+name for name in names], test_loader, scaler)

100%|██████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 59.06 batch/s]


In [50]:
df2 = getParameterLoss(single_fcs, ["Multi LSTM "+name for name in names], test_loader, scaler)

100%|██████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 56.39 batch/s]


In [51]:
df

,YM (Skin),YM (Adipose),PR (Skin),PR (Adipose),Perm (Skin),Perm (Adipose),Overall
Architecture,,,,,,,
Single LSTM Output,96.525055,85.785454,99.828110,99.822449,84.821022,95.499870,93.713660
Single LSTM Hidden,96.348923,86.934082,99.814545,99.831612,84.679314,95.509895,93.853059
Single LSTM Final Hidden,94.659325,85.610786,99.734825,99.805794,82.297386,95.269272,92.896231
Single LSTM Final Output,95.141014,87.076744,99.777969,99.824455,79.363625,96.451836,92.939274
Single LSTM Final Output + Final Hidden,95.375092,86.844162,99.781479,99.837769,85.613510,95.484497,93.822752


In [52]:
df2

,YM (Skin),YM (Adipose),PR (Skin),PR (Adipose),Perm (Skin),Perm (Adipose),Overall
Architecture,,,,,,,
Multi LSTM Output,96.634399,85.943565,99.830055,99.835670,84.929535,95.505203,93.779739
Multi LSTM Hidden,96.384247,85.413345,99.812164,99.835358,85.829903,95.093933,93.728160
Multi LSTM Final Hidden,95.093628,85.438332,99.768372,99.797066,85.802467,95.246353,93.524369
Multi LSTM Final Output,96.085976,87.055252,99.805824,99.845558,87.441376,96.494759,94.454791
Multi LSTM Final Output + Final Hidden,95.128815,86.760498,99.764488,99.835655,85.643105,95.406212,93.756462


In [55]:
df.to_csv("../Results/single_LSTM.csv")

In [56]:
df2.to_csv("../Results/multi_LSTM.csv")